In [80]:
!pip install demoji
!pip install num2words

In [81]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [144]:
from num2words import num2words
import demoji
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
wordnet_lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [2]:
descriptionsFile = pd.read_csv('MyAppDescriptions.csv')

In [189]:
def numberToWord(description):
    split = description.split()
    for i in range(len(split)): 

        if (split[i].isdigit()):
            split[i] = num2words(split[i])

    return " ".join(split)

In [193]:
stop_words = set(stopwords.words('english'))
def removeStopWords(description):
    word_tokens = word_tokenize(description) 
    return [w for w in word_tokens if not w in stop_words]

In [194]:
def stemmatize(description):
    stemmed = []
    for w in description: 
        stemmed.append(stemmer.stem(w, True))
    return " ".join(stemmed)

In [200]:
def lemmatize(description):
    lemmatized = []
    for w in description: 
        lemmatized.append(wordnet_lemmatizer.lemmatize(w))
    return " ".join(lemmatized)

## Preprocess
please uncomment to see the results gotten after every filter operation

In [218]:
data = []
for i in range(len(descriptionsFile)):
    app = descriptionsFile.loc[i]
    description = app['App Description']
#     print(description)
    punct_removed = description.translate(str.maketrans('', '', string.punctuation))
#     print(punct_removed)
    backslash_rm = re.sub("\n", " ", punct_removed )
#     print(backslash_rm)
    noEmo = demoji.replace(backslash_rm, '')
    filtered = re.sub("[✦★❖•●’]", "", noEmo)
#     print(filtered)
    string_only = numberToWord(filtered).lower()
#     print(string_only)
    sw_removed = removeStopWords(string_only)
#     print(sw_removed)
    stemmatized = stemmatize(sw_removed)
#     print(stemmatized)
    lemmatized = lemmatize(sw_removed)
    print(lemmatized)
    data.append([app['Package Name'],app['App Name'],lemmatized])


stayfree stayfree screen time tracker limit app usage self control productivity phone addiction controller app allows show much time spend smartphone help focus restricting usage apps set usage limit apps receive alert exceeding usage limit also view detail usage statistic usage history make stayfree special highest rated screen time self control app extremely fast userfriendly interface accurate usage statistic crossplatform usage statistic analysis quick customer support battery friendly totally adfree stayfree screen time tracker limit app usage help overcome phone addiction stay focused selfcontrol reduce screen time reduce distraction boost productivity unplug often increase digital wellbeing spend quality time family reduce wasted time digital detox apps feature app usage history view chart statistic usage history overuse reminder notify spending much time app start digital detox block mode temporarily block application overusing crossplatform stayfree multiplatform save time dev

people becoming addicted mobile phone whether adult child whether time dinner party problem mobile addiction become serious people simply dont realize spend lot time every day apps game using screen time better understand usage mobile phone whether game app install screen time person time management awareness successful person manage time better usually dont realize spend lot time application screen time make aware habit using phone make adjustment accordingly maybe watched many video maybe spent much time social network app daily usage screen time show detailed view daily mobile phone usage accurate use phone hour apps opened long used screen time better allocate mobile phone usage time screen time know duration hour use type app used app weekly usage week app usage checking statistic mobile phone usage last week know daily mobile phone usage trend app category limit set daily duration limit app type app set different duration day weekend game usage time set different time limit day i

In [219]:
ppDescriptionsDF = pd.DataFrame(data, columns=['Package Name', 'App Name', 'App Description'])
ppDescriptionsDF.to_csv("./PreProcessedDescription.csv")

## Difference between stemming and lemmatization?
Stemming - Stemming is a process of reducing words to its root form even if the root has no dictionary meaning. For eg: beautiful and beautifully will be stemmed to beauti which has no meaning in English dictionary.

Lemmatisation - Lemmatisation is a process of reducing words into their lemma or dictionary. It takes into account the meaning of the word in the sentence. For eg: beautiful and beautifully are lemmatised to beautiful and beautifully respectively without changing the meaning of the words. But, good, better and best are lemmatised to good since all the words have similar meaning.

## Bigrams and Trigrams

In [223]:
preProcessedFile = pd.read_csv('PreProcessedDescription.csv')

def getBigrams(description):
    tokens = [token for token in description.split(" ") if token != ""]
    return list(ngrams(tokens, 2))   
        
def getTrigrams(description):
    tokens = [token for token in description.split(" ") if token != ""]
    return list(ngrams(tokens, 3))   
 
bigramsData = []
trigramsData = []

for i in range(len(preProcessedFile)):
        
    app = preProcessedFile.loc[i]
    bigrams = getBigrams(app['App Description'])        
    trigrams = getTrigrams(app['App Description'])

    bigramsList = []
    for w in bigrams:
        row = ','.join(w)
        bigramsList.append(row)
        
    #list of bigrams seperated by a whitespace makes its very easy to store them and split them when needed 
    bis = ' '.join(bigramsList)
#     print(bis + '\n\n\n')
    bigramsData.append([app['Package Name'], app['App Name'], bis])
    
    trigramsList = []
    for w in trigrams:
        row = ','.join(w)
        trigramsList.append(row)

    tris = ' '.join(trigramsList)
#     print(tris + '\n\n\n')
    trigramsData.append([app['Package Name'], app['App Name'], tris])

    

In [249]:
biFile = pd.DataFrame(bigramsData, columns=['Package Name', 'App Name', 'Bigrams'])
biFile.to_csv("./Bigrams.csv")
triFile = pd.DataFrame(trigramsData, columns=['Package Name', 'App Name', 'Trigrams'])
triFile.to_csv("./Trigrams.csv")

## COMPARE FEATURES

In [258]:
appFeaturesFile = pd.read_csv('MyAppFeatures.csv')

# calculate hits and losses for each app
# check if the bigram exists in the feature extracted manually
# if it does then hit++, else loss++

for i in range(len(biFile)):
    app1 = biFile.loc[i]
    bigrams = app1['Bigrams']
    bigramsList = []

    #format bigrams remove , to make substring comparisons    
    for bigram in bigrams.split():
        bi = " ".join(bigram.split(','))
        bigramsList.append(bi)
    
    hits = 0
    losses = 0
    for i in range(len(appFeaturesFile)):
        app = appFeaturesFile.loc[i]
        if (app['Package Name'] == app1['Package Name']):
            feature = app['Feature'].lower()
            for bigram in bigramsList:
                if(bigram in feature): 
                    hits=hits+1                 
                else:
                    losses=losses + 1
    
    print(app1['App Name'])
    print(str(hits) + ' hits and ' + str(losses) + ' losses. H/L ratio: ' + str(hits/losses) + '\n')



StayFree - Stay Focused
57 hits and 3869 losses. H/L ratio: 0.014732489015249418

Screen Time - Stay Focused
173 hits and 2578 losses. H/L ratio: 0.06710628394103957

OffScreen - Less Screen Time
46 hits and 908 losses. H/L ratio: 0.05066079295154185

Digitox : Screen Time
26 hits and 2494 losses. H/L ratio: 0.010425020048115477

Screen Time
43 hits and 602 losses. H/L ratio: 0.07142857142857142

Digital Wellbeing
33 hits and 541 losses. H/L ratio: 0.06099815157116451

Screen Time Digital Wellbeing
38 hits and 1516 losses. H/L ratio: 0.025065963060686015

Screen Time - Restrain yoursel
56 hits and 1138 losses. H/L ratio: 0.0492091388400703

ActionDash: Screen Time Helper
33 hits and 1276 losses. H/L ratio: 0.02586206896551724

YourHour - ScreenTime Control
61 hits and 2347 losses. H/L ratio: 0.025990626331487004



In [259]:
for i in range(len(triFile)):
    app1 = triFile.loc[i]
    trigrams = app1['Trigrams']
    trugramsList = []

    #format trigrams remove , to make substring comparisons
    for trigram in trigrams.split():
        tri = " ".join(trigram.split(','))
        trigramsList.append(tri)
    
    hits = 0
    losses = 0
    for i in range(len(appFeaturesFile)):
        app = appFeaturesFile.loc[i]
        if (app['Package Name'] == app1['Package Name']):
            feature = app['Feature'].lower()
            for trigram in trigramsList:
                if(trigram in feature): 
                    hits=hits+1                 
                else:
                    losses=losses + 1
    print(app1['App Name'])
    print(str(hits) + ' hits and ' + str(losses) + ' losses. H/L ratio: ' + str(hits/losses) + '\n')
    


StayFree - Stay Focused
48 hits and 121697 losses. H/L ratio: 0.0003944222125442698

Screen Time - Stay Focused
175 hits and 68124 losses. H/L ratio: 0.002568845047266749

OffScreen - Less Screen Time
45 hits and 88713 losses. H/L ratio: 0.0005072537283149031

Digitox : Screen Time
20 hits and 91249 losses. H/L ratio: 0.00021918048416968953

Screen Time
35 hits and 51310 losses. H/L ratio: 0.0006821282401091405

Digital Wellbeing
51 hits and 72399 losses. H/L ratio: 0.0007044296191936353

Screen Time Digital Wellbeing
40 hits and 73957 losses. H/L ratio: 0.0005408548210446611

Screen Time - Restrain yoursel
95 hits and 64519 losses. H/L ratio: 0.0014724344766657883

ActionDash: Screen Time Helper
60 hits and 76625 losses. H/L ratio: 0.0007830342577487765

YourHour - ScreenTime Control
75 hits and 89965 losses. H/L ratio: 0.0008336575334852443



## Discussion
Based on this little experiment done above, bigrams seem to perform much better than trigrams when compared to the app features
If we visually inspect the bigrams and trigrams by eye, they look like they have captured a lot about the essence of what the application does. However when compared to the manually extracted features which were just copied and pasted from the description of the app, and had no preprocessing done on them, the comparisons lead to not a very good hit/loss ratio for both bigrams and trigrams.
But maybe that was because of the way I was comparing or the way I was calculating the losses.
Overall, I think Bi-Grams and Tri-Grams are reliable for getting the gist of an app even though the hit/loss ratio i got was not very good. I say that because when going through the list of biandtrigrams I was able to understand what features the app has.